In [ ]:
from balm.data import load_dataset, DataCollator
from balm.models import (
    BalmForMaskedLM,
    BalmMoEForMaskedLM,
    BalmExpertChoiceMoEForMaskedLM,
    BalmHybridMoEForMaskedLM,
)
from balm.tokenizer import Tokenizer
from balm.train import Trainer

In [ ]:
tokenizer = Tokenizer(vocab="./vocab.json")

In [ ]:
def remove_sep(txt):
    return txt.replace("</s>", "<cls><cls>")


data_files = {
    "train": "./balm/test_data/test.txt",
    "test": "./balm/test_data/test_1k.txt",
    "eval": "./balm/test_data/test_1k.txt",
}

dataset = load_dataset("text", data_files=data_files, preprocess_fn=remove_sep)

In [ ]:
tokenized_dataset = dataset.map(
    lambda x: tokenizer(
        x["text"],
        padding=True,
        truncation=True,
        max_length=320,
    ),
    remove_columns="text"
)

In [ ]:
collator = DataCollator(tokenizer=tokenizer)

In [11]:
# matched to ESM-2 8M
model = BalmForMaskedLM(
    embed_dim=320,
    ffn_dim=320*4,
    num_layers=6,
    num_heads=20,
    vocab_size=tokenizer.vocab_size,
)

# # matched to ESM-2 8M
# model = BalmMoEForMaskedLM(
#     embed_dim=320,
#     ffn_dim=320*4,
#     num_experts=8,
#     num_shared_experts=0,
#     num_layers=6,
#     num_heads=20,
#     alternate_sparsity=True,
#     router_top_k=1,
#     expert_capacity=128,
#     router_z_loss_coef=0.01,
#     router_aux_loss_coef=0.01,
#     vocab_size=tokenizer.vocab_size,
# )

# # matched to ESM-2 8M
# model = BalmExpertChoiceMoEForMaskedLM(
#     embed_dim=320,
#     ffn_dim=320 * 4,
#     num_experts=8,
#     num_shared_experts=0,
#     num_layers=6,
#     num_heads=20,
#     alternate_sparsity=False,
#     expert_capacity=128,
#     router_z_loss_coef=0.01,
#     vocab_size=tokenizer.vocab_size,
# )

In [12]:
model.num_parameters

6294113

In [13]:
trainer = Trainer(
    model=model,
    data_collator=collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["eval"],
    epochs=1,
    logging_steps=10,
    eval_steps=50,
    warmup_steps=50,
    per_device_train_batch_size=32,
    # use_cpu=True,
)

In [14]:
trainer.device

device(type='cpu')

In [15]:
trainer.train()

Training:   0%|          | 0/2087 [00:00<?, ?step/s]

step 10   | loss: 3.1020 | lr: 0.000080
step 20   | loss: 2.7443 | lr: 0.000160
step 30   | loss: 2.6527 | lr: 0.000240
step 40   | loss: 2.6453 | lr: 0.000320
step 50   | loss: 2.5886 | lr: 0.000400


Evaluating:   0%|          | 0/31 [00:00<?, ?step/s]

<<< EVAL >>> loss: 2.6825
step 60   | loss: 2.5415 | lr: 0.000398
step 70   | loss: 2.2838 | lr: 0.000396
step 80   | loss: 2.2063 | lr: 0.000394
step 90   | loss: 2.1203 | lr: 0.000392
step 100  | loss: 2.1287 | lr: 0.000390


Evaluating:   0%|          | 0/31 [00:00<?, ?step/s]

...

Evaluating:   0%|          | 0/31 [00:00<?, ?step/s]

<<< EVAL >>> loss: 0.4009
step 1760 | loss: 0.3021 | lr: 0.000064
step 1770 | loss: 0.3456 | lr: 0.000062
step 1780 | loss: 0.3505 | lr: 0.000060
step 1790 | loss: 0.4145 | lr: 0.000058
step 1800 | loss: 0.3243 | lr: 0.000056


Evaluating:   0%|          | 0/31 [00:00<?, ?step/s]

<<< EVAL >>> loss: 0.3980
step 1810 | loss: 0.3906 | lr: 0.000054
step 1820 | loss: 0.3631 | lr: 0.000052
step 1830 | loss: 0.3979 | lr: 0.000050
step 1840 | loss: 0.3852 | lr: 0.000049


KeyboardInterrupt: 